In [1]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask

import pandas as pd
import time

from functools import partial
print = partial(print,flush=True)

import multiprocessing as mp
import numpy as np
from itertools import combinations

pd.options.display.max_rows = 999

np.seterr(divide='ignore')
ProgressBar(dt=5).register()

In [2]:
def vif_core(i, X):
    return X.columns[i], vif(X.values, i)

def make_vif(X):
    with mp.Pool(processes=7) as pool:
        vif_data = {}
        for index, (col_name, vif_value) in enumerate(pool.imap_unordered(partial(vif_core,X=X),range(X.shape[1]))):
            vif_data[col_name] = vif_value
            if (index % int(X.shape[1]/50)) == 0:
                print((index+1)/X.shape[1])
    
    vif_data = pd.Series(vif_data)
    return vif_data.sort_values(ascending=False)

In [3]:
X = dd.read_csv("./data/ready_for_models2.csv/*.part")

X = X.sample(frac=.01).compute().astype(float)

# permission_denied = X['permission_denied']
X = X.drop("permission_denied",axis=1)
X = add_constant(X)


[########################################] | 100% Completed |  3min 16.0s


In [5]:
v = make_vif(X)

0.01282051282051282
0.02564102564102564
0.038461538461538464
0.05128205128205128
0.0641025641025641
0.07692307692307693
0.08974358974358974
0.10256410256410256
0.11538461538461539
0.1282051282051282
0.14102564102564102
0.15384615384615385
0.16666666666666666
0.1794871794871795
0.19230769230769232
0.20512820512820512
0.21794871794871795
0.23076923076923078
0.24358974358974358
0.2564102564102564
0.2692307692307692
0.28205128205128205
0.2948717948717949
0.3076923076923077
0.32051282051282054
0.3333333333333333
0.34615384615384615
0.358974358974359
0.3717948717948718
0.38461538461538464
0.3974358974358974
0.41025641025641024
0.4230769230769231
0.4358974358974359
0.44871794871794873
0.46153846153846156
0.47435897435897434
0.48717948717948717
0.5
0.5128205128205128
0.5256410256410257
0.5384615384615384
0.5512820512820513
0.5641025641025641
0.5769230769230769
0.5897435897435898
0.6025641025641025
0.6153846153846154
0.6282051282051282
0.6410256410256411
0.6538461538461539
0.6666666666666666
0.

In [6]:
v

adverbAdverbs                  4.490693
osocialSocial                  4.443012
discrepDiscrepancies           4.289963
progm                          4.133257
auxverbAuxiliaryVerbs          4.117237
negateNegations                3.912586
differDifferentiation          3.538933
modal_pa                       3.483011
general_pa                     3.428911
assentAssent                   3.352917
insightInsight                 3.278323
conjConjunctions               2.996799
spaceSpace                     2.849777
prepPrepositions               2.703723
onegemoNegativeEmotions        2.471496
quanunit                       2.392337
posemoPositiveEmotions         2.387820
sheheSheHe                     2.292063
quantQuantifiers               2.222704
powerPower                     2.214680
achieveAchievement             2.202978
numberNumbers                  2.175742
workWork                       2.170510
compareComparisons             2.128699
maleMale                       2.119416
